In [31]:
import requests 
import time
from bs4 import BeautifulSoup as bs
from urllib.error import URLError
import re
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import os
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False
url = 'http://sh.bendibao.com/ditie/linemap.shtml'
#get dict_lines ,each lines dict with station names
response = requests.get(url)
soup = bs(response.text,"lxml")
soup_lines= soup.find_all("div",class_="line-list")
dict_lines = {}
for cont in soup_lines:
    line_name = cont.find("a",href=re.compile("ditie/map")).get_text()
    station = []
    station_doc = cont.find_all('a',href=re.compile("/ditie/zd"))
    for cont_station in station_doc:
        station.append(cont_station.get_text())
    dict_lines[line_name] = station 
#get dict_lines_loc, each line with each dot longitude and latitude
what_we_want = r'<a href="(/ditie/map_\d+\.shtml)" target="_blank">(\w+)</a></strong>'
pattern2 = re.compile(what_we_want)
lines = pattern2.findall(response.text)
dict_lines_loc = defaultdict(list)
for line_info in lines:
    line_url = 'http://sh.bendibao.com/' + list(line_info)[0]
    response = requests.get(line_url)
    sub_url_info = bs(response.text,"lxml")
    tmp_info = sub_url_info.find(id="frmMap")
    location_str = tmp_info.get('src')
    location_list = re.findall(r'jd(\d+)=(\d+\.\d+)&wd\d+=(\d+.\d+)',location_str)
    dict_lines_loc[list(line_info)[1]] = location_list
f = open('SH_MRT_station.txt','w+')
f.write(str(dict_lines))
f.close()
dict_lines_tmp = dict_lines['上海地铁9号线线路图']
dict_lines_loc_tmp = dict_lines_loc['上海地铁9号线线路图']
# some station have no longtitude and latitude
dict_reloc = {}
for k,v in dict_lines.items():
    if len(dict_lines[k])==len(dict_lines_loc[k]) :
        for i,j in enumerate(v):
            dict_lines_loc[k][i] = list(dict_lines_loc[k][i])
            dict_lines_loc[k][i][0] = dict_lines[k][i]
            dict_reloc[dict_lines[k][i]] = dict_lines_loc[k][i][1:]
    elif len(dict_lines[k]) > len(dict_lines_loc[k]) :
        for i,j in enumerate(dict_lines_loc[k]):
            m = int(dict_lines_loc[k][i][0])
            dict_lines_loc[k][i] = list(dict_lines_loc[k][i])
            dict_lines_loc[k][i][0] = dict_lines[k][m]
            dict_reloc[dict_lines[k][m]] = tuple(dict_lines_loc[k][i][1:])
    elif len(dict_lines_loc[k]) == 0 :
        del dict_lines[k]
        del dict_lines_loc[k]

f = open('SH_MRT2.txt','w+')
f.write(str(dict_reloc))
f.close()
